## Multiprocessing

- Pythonの言語仕様でサポートされている機能
- オペレーティングシステムが提供するプロセスのインタフェース（システムコール）に近い
  - システムコールは os モジュールから呼び出し可能
- 汎用性は高い一方で、使いやすくはない

multiprocessing モジュールは JupyterLab と一緒に使えないため、以下では同様のインタフェースを提供する multiprocess を用いる。

### インストール

（2022年度）
multiprocess をインストールすると Python 本体も conda-forge からインストールされてしまう。
conda-forge の Python を用いる影響が不明のため、仮想環境を作成してインストールする。

```
conda create -n multiprocess -c conda-forge python=3.9 jupyterlab multiprocess
```
https://anaconda.org/conda-forge/multiprocess

### プロセスの生成

In [ ]:
from multiprocess import Process
import os

# プロセスIDの表示
def do_print(str):
    print('pid({}) Process {}'.format(os.getpid(), str))

# 親プロセス（自分）のプロセスID
do_print('Parent')

# プロセス制御オブジェクトの作成
# - target: 子プロセスで実行する関数
# - args: 引数のタプル ('Child',) と , の後に何もないのは要素が1つのタプルであるため
p = Process(target=do_print, args=('Child',))

# 子プロセスの実行開始
p.start()

print('Wating for child process...')

# 子プロセスの終了待機
p.join()

### 複数プロセスの生成

- 複数のプロセスからなるプロセスプール（ワーカプロセス）を作成
- 作成したプロセスで関数を実行、結果を受け取る

以下は Windows では実行不可（Mac, Linux では可能）

In [ ]:
from multiprocess import Pool
from random import randint

# 子プロセスに実行させる関数
# - 1からランダムに生成された整数までの総和を計算
def do_some_work(pid):
    n = randint(100, 10000)
    print('pid({}) Process #{} work({})'.format(os.getpid(), pid, n))
    return (pid, sum(range(1, n + 1)))

# プロセスプールの作成
# - 引数は作成するプロセス数
p = Pool(4)

# プロセスプールのプロセスに仕事を割り当てる
# - 第2引数のイテラブルが消費されるまで割り当てを続ける
result = p.map_async(do_some_work, range(10))

# 結果を受け取る
# - timeout=None: 例外を発生させるタイムアウトを設定しない
result.get(timeout=None)